Kaiwen provided the code to convert cine short axis dicom images to mp4 gif files. Mahri compiled all the data together, obtained gif files, and put all the necessary filepaths into scd_patientdata_xlsx excel file

In [ ]:
import os
import pydicom
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, FFMpegWriter
import matplotlib.animation as animation
import pandas as pd

In [ ]:
# User handle where the SCD_Images folders are located
user_handle = r"C:\Users\mkkad\Desktop\github_repos\heart_cardiac_mri_image_processing\data"  

In [ ]:
# Define dataframe of patient data and cine short axis images filepath
patient_data = pd.read_excel("scd_patientdata_xlsx.xlsx")

# Get the absolute filepath to cine short axis mri images for each patient
abs_filepath = [os.path.join(user_handle, i) for i in patient_data['filepath'].tolist()]

In [ ]:
# Obtain a dictionary where keys are patient filepaths and values of each key are dicom image filepaths
patient_dicom_files = {}

for filepath in abs_filepath:

    dicom_files = [os.path.join(filepath, i) for i in os.listdir(filepath) if i.endswith(".dcm")]

    patient_dicom_files[filepath] = dicom_files

In [ ]:
# Create the gif given the dicom images list and output filepath with .gif extension
def create_dicom_gif(images, output_filepath, fps=2):
    if not images:
        print("❌ No DICOM images to process.")
        return

    fig, ax = plt.subplots()
    img_display = ax.imshow(images[0], cmap="gray")
    ax.axis("off")

    # Update function for animation
    def update(frame):
        img_display.set_data(images[frame])
        ax.set_title(f"Frame {frame + 1} of {len(images)}")

    ani = animation.FuncAnimation(
        fig, update, frames=len(images), interval=500  # 500 ms delay = 2 FPS
    )

    # Save the animation as an MP4 file
    ani.save(output_filepath, writer="pillow", fps=fps)
    plt.close(fig)
    print(f"🎥 gif saved as: {output_filepath}")

In [ ]:
# Generate mp4 gif for every patient and save the gif in their folders
gif_filepaths = []
for idx, (patient, patient_files) in enumerate(patient_dicom_files.items()):


    patient_gif_name = os.path.basename(os.path.dirname(patient)) + "_gif.gif"
    patient_gif_filepath = os.path.join(os.path.dirname(patient), patient_gif_name)
    
    patient_gif_filepath_to_append = os.path.join(os.path.dirname(patient_data['filepath'].tolist()[idx]), patient_gif_name)

    gif_filepaths.append(patient_gif_filepath_to_append)


    patient_dicom_images = []
    for img_path in sorted(patient_files):
        try:
            dicom_data = pydicom.dcmread(img_path)
            patient_dicom_images.append(dicom_data.pixel_array)
        except Exception as e:
            print(f"⚠️ Skipping {img_path} due to error: {e}")
    
    create_dicom_gif(patient_dicom_images, output_filepath=patient_gif_filepath, fps=20)
    

In [ ]:
# Write into dataframe and save it as excel file
patient_data['gif_filepath'] = gif_filepaths
patient_data
patient_data.to_excel("scd_patientdata_xlsx.xlsx", index=False)